# Importing the Libraries

In [ ]:
import time
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import array
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import posenet

# This is the posedetector function which helps us to detect pose

In [ ]:

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368                                            # setting up the
                                                        # deafult values
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

def poseDetector(image):
    imageWidth = image.shape[1]
    imageHeight = image.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(image, 1.0, (width, height), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  #basically its just here because we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # taking heatmap of respective body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (imageWidth* point[0]) / out.shape[3]
        y = (imageHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        From = pair[0]
        To = pair[1]
        assert(From in BODY_PARTS)
        assert(To in BODY_PARTS)

        idFrom = BODY_PARTS[From]
        idTo = BODY_PARTS[To]

        if points[idFrom] and points[idTo]:
            cv.line(image, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(image, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(image, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()

    return image 

# This is for accesing and preprocessing the training and testing files

In [ ]:
poses = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
feature=[]
feature_test=[]
labels = []
labels_test=[]
def create_train(DIR,label,feat):
    for person in poses:
        path = os.path.join(DIR, person)
        for img in os.listdir(path):
            img_path = os.path.join(path,img)
            img_array = cv.imread(img_path)
            if img_array is None:
                continue 
            else:
                frame = poseDetector(img_array)
                frame_new=cv.imread(img_path)
                if(frame.shape[2]==frame_new.shape[2]):
                    frame_diff=frame-frame_new
                    feat.append(frame_diff)
                    label.append(person)

start=time.time()
pose=create_train(r'C:\Users\Kuwar\python projects\yoga pose\Train',labels,feature)
features=np.array(feature)
pose_t = create_train(r'C:\Users\Kuwar\python projects\yoga pose\Test',labels_test,feature_test)
features_test=np.array(feature_test)
end=time.time()
print('preprocessing done at:- ')
print({end-start})

# This part also mainly focuses on shaping up the data 

In [ ]:
features.reshape(len(labels),1)
features_test.reshape(len(labels_test),1)
print(features.shape)
features_test.shape

In [ ]:
data_test=pd.DataFrame(features_test ,columns=['feature_t'])
data_test1=pd.DataFrame(labels_test ,columns=['label_t'])
data=pd.DataFrame(features ,columns=['feature'])
data1=pd.DataFrame(labels ,columns=['label'])

In [ ]:
for i in range(len(data['feature'])):
    data['feature'][i]=data['feature'][i].flatten()
for i in range(len(data_test['feature_t'])):
    data_test['feature_t'][i]=data_test['feature_t'][i].flatten()

# Label Encoder

In [ ]:
lr=LabelEncoder()
labels=lr.fit_transform(data1['label'])
labels_test=lr.fit_transform(data_test1['label_t'])
labels.shape

# Storing The Data for classification

In [ ]:
train3=[]
test3=[]
for i in range(len(data['feature'])):
     train3.append(data['feature'][i].std())
for i in range(len(data_test['feature_t'])):
     test3.append(data_test['feature_t'][i].std())
        
train3=np.array(train3)
train3=np.reshape(train3,(len(labels),1))
test3=np.array(test3)
test3=np.reshape(test3,(len(labels_test),1))

# KNeighbor Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train3,labels)
predict_knn=knn.predict(test3)
print(classification_report(labels_test,predict_knn))
print(confusion_matrix(labels_test,predict_knn))
knn.fit(train3,labels)
predict_knn1=knn.predict(train3)
print(classification_report(labels,predict_knn1))
print(confusion_matrix(labels,predict_knn1))